In [1]:
# @tool 데코레이터로 랭체인에 함수 연결하기
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
llm = ChatOpenAI(model="gpt-4o-mini")

llm.invoke([HumanMessage("잘 지냈어?")])

AIMessage(content='잘 지냈어요! 당신은 어떻게 지내고 계신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 12, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bp3mA3aNqtzqQFaysDdseHpqfFl9T', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--45679990-d125-4c7b-9f4d-ad2f10271cc9-0', usage_metadata={'input_tokens': 12, 'output_tokens': 16, 'total_tokens': 28, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [2]:
# get_current_time 함수를 랭체인으로 언어 모델에 연결
# 함수에 """ """ 로 표시된 설명은 단순한 주석이 아니라 랭체인에 이 함수의 기능과 입력값, 사용 방법을 알려주는 문서화 문자열(docsstring)입니다.
# 펑션 콜링에서는 딕셔너리로 정보를 작성했지만 이제 문서화 문자열로 작성합니다.

from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool # @tool 데코레이터를 사용하여 함수를 도구로 등록
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수

    Args:
        timezone (str): 타임존 (예: 'Asia/Seoul') 실제 존재하는 타임존이어야 함
        location (str): 지역명. 타임존이 모든 지명에 대응되지 않기 때문에 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재시각 {now} ' # 타임존, 지역명, 현재시각을 문자열로 반환
    print(location_and_local_time)
    return location_and_local_time

In [3]:
# 도구를 tools 리스트에 추가하고, tool_dict에도 추가
tools = [get_current_time,]
tool_dict = {"get_current_time": get_current_time,}

# 도구를 모델에 바인딩: 모델에 도구를 바인딩하면, 도구를 사용하여 llm 답변을 생성할 수 있음
llm_with_tools = llm.bind_tools(tools)

In [4]:
# 사용자 질문과 도구를 사용 언어 모델 답변 생성
from langchain_core.messages import SystemMessage

# (4) 사용자의 질문과 tools 사용하여 llm 답변 생성
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?"),
]

# (5) llm_with_tools를 사용하여 사용자의 질문에 대한 llm 답변 생성
response = llm_with_tools.invoke(messages)
messages.append(response)

# (6) 생성된 llm 답변 출력
print(messages)

[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5W4qTe3hVo1E1nlydfe7vUEW', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"부산"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 135, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bp3mBY7DXYeclDqhJak94pc9QJVts', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e52126ac-6ad1-40b8-85dd-389fb8073316-0', tool_calls

In [5]:
# 여러 도시를 물을 경우 get_current_time 함수를 여러 번 호출 (for 활용)
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]] # (7) tool_dict를 사용하여 도구 함수를 선택
    print(tool_call["args"]) # (8) 도구 호출 시 전달된 인자 출력
    tool_msg = selected_tool.invoke(tool_call) # (9) 도구 함수를 호출하여 결과를 반환
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul (부산) 현재시각 2025-07-03 11:15:44 


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5W4qTe3hVo1E1nlydfe7vUEW', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"부산"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 135, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bp3mBY7DXYeclDqhJak94pc9QJVts', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e52126ac-6ad1-40b8-85dd-389fb8073316-0', tool_cal

In [6]:
# 함수 실행 결과를 문장으로 출력하기
llm_with_tools.invoke(messages)

AIMessage(content='부산은 현재 2025년 7월 3일 11시 15분 44초입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 192, 'total_tokens': 218, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bp3mCJPfWg9lRXjgn5eIbJaZYMYjm', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--418ab053-92d3-4cbe-834b-8f87ae552b8c-0', usage_metadata={'input_tokens': 192, 'output_tokens': 26, 'total_tokens': 218, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
# Pydantic은 입력된 데이터의 유효성과 형식을 검증하고 특정 데이터 형식으로 명확하게 표현할 때 사용하는 라이브러리
from pydantic import BaseModel, Field

class StockHistoryInput(BaseModel):
    ticker: str = Field(..., title="주식 코드", description="주식 코드 (예: AAPL)")
    period: str = Field(..., title="기간", description="주식 데이터 조회 기간 (예: 1d, 1mo, 1y)")

In [8]:
import yfinance as yf

@tool
def get_yf_stock_history(stock_history_input: StockHistoryInput) -> str:
    """ 주식 종목의 가격 데이터를 조회하는 함수"""
    stock = yf.Ticker(stock_history_input.ticker)
    history = stock.history(period=stock_history_input.period)
    history_md = history.to_markdown()

    return history_md

# GPT에게 어떤 툴이 있는지 알려줌
tools = [get_current_time, get_yf_stock_history]
# GPT가 툴을 실제 호출하려 할 때 필요
tool_dict = {
    "get_current_time": get_current_time,
    "get_yf_stock_history": get_yf_stock_history,
}

llm_with_tools = llm.bind_tools(tools)

In [9]:
messages.append(HumanMessage("테슬라는 한 달 전에 비해 주가가 올랐나 내렸나?"))

response = llm_with_tools.invoke(messages)
print(response)
messages.append(response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_lyNRWR9JEVea2iObSv3lD0ES', 'function': {'arguments': '{"stock_history_input":{"ticker":"TSLA","period":"1mo"}}', 'name': 'get_yf_stock_history'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 283, 'total_tokens': 310, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-Bp3mDGkGKrIvhYzvpcnAoVpFkzMwz', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--44186bd6-fb72-4bd8-9728-b04ab04e564c-0' tool_calls=[{'name': 'get_yf_stock_history', 'args': {'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}, 'id': 'call_lyNRWR9JEVea2iObSv3lD0ES', 'type': 'tool_call'}] usage_metadata={'inp

In [10]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    print(tool_msg)

{'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}
content='| Date                      |   Open |   High |    Low |   Close |      Volume |   Dividends |   Stock Splits |\n|:--------------------------|-------:|-------:|-------:|--------:|------------:|------------:|---------------:|\n| 2025-06-03 00:00:00-04:00 | 346.6  | 355.4  | 343.04 |  344.27 | 9.93245e+07 |           0 |              0 |\n| 2025-06-04 00:00:00-04:00 | 345.1  | 345.6  | 327.33 |  332.05 | 9.89121e+07 |           0 |              0 |\n| 2025-06-05 00:00:00-04:00 | 322.49 | 324.55 | 273.21 |  284.7  | 2.875e+08   |           0 |              0 |\n| 2025-06-06 00:00:00-04:00 | 298.83 | 305.5  | 291.14 |  295.14 | 1.64748e+08 |           0 |              0 |\n| 2025-06-09 00:00:00-04:00 | 285.96 | 309.83 | 281.85 |  308.58 | 1.40909e+08 |           0 |              0 |\n| 2025-06-10 00:00:00-04:00 | 314.94 | 327.83 | 310.67 |  326.09 | 1.51256e+08 |           0 |              0 |\n| 2025-06-11 00:00:00-04:0

In [11]:
# 자연어로 함수 결과 처리하기
llm_with_tools.invoke(messages)

AIMessage(content='한 달 전 테슬라(TSLA)의 주가는 다음과 같았습니다:\n\n- **2025년 6월 3일**: 종가 344.27\n\n현재 날짜인 **2025년 7월 2일**의 테슬라 주가는 **315.65**입니다.\n\n비교 결과:\n- 한 달 전 주가: **344.27**\n- 현재 주가: **315.65**\n\n결론적으로, 테슬라의 주가는 한 달 동안 내렸습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 1588, 'total_tokens': 1696, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-Bp3mFKcBavZ26anOqCGYeVF2vgoiW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--bfa3e519-ecfe-4933-9c8c-62ea638e58e7-0', usage_metadata={'input_tokens': 1588, 'output_tokens': 108, 'total_tokens': 1696, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reason

In [12]:
# .stream 함수 출력 확인
# 툴 없이 언어 모델만 있을 경우 스트리밍 출력하기
for c in llm.stream([HumanMessage("잘 지냈어? 한국 사회의 문제점이 무엇인지 이야기해줘."),]):
    print(c.content, end="|")

|안|녕하세요|!| 한국| 사회|에는| 여러| 가지| 복|잡|한| 문제|점|이| 존재|합니다|.| 몇| 가지| 주요| 문제|를| 살|펴|보|면|:

|1|.| **|고|용|문|제| 및| 청|년| 실|업|**|:| 청|년|층|의| 실|업|률|이| 높|고|,| 안정|적인| 일|자|리를| 찾|기| 어려|운| 상황|입니다|.| 특히| 비|정|규|직|,| 공|공|부|문| 경쟁| 등|으로| 인해| 경쟁|이| 치|열|합니다|.

|2|.| **|주|택| 문제|**|:| 부|동|산| 가격| 상승|과| 전|세|난|이| 심|각|한| 문제|로|,| 많은| 사람들이| 주|거| 불|안을| 느|끼|고| 있습니다|.| 특히| 대|도|시|에서|의| 주|택| 구매|가| 어려|운| 사례|가| 많|습니다|.

|3|.| **|사회|적| 불|평|등|**|:| 소|득| 불|균|형|이| 심|화|되고| 있으며|,| 대|기업|과| 중|소|기업| 간|의| 격|차|,| 지역| 간| 불|균|형| 등| 다양한| 형태|의| 불|평|등|이| 존재|합니다|.

|4|.| **|고|령|화| 사회|**|:| 인|구| 고|령|화|로| 인해| 노|인| 복|지|와| 관련|된| 문제가| 증가|하고| 있습니다|.| 이는| 노동|력| 부족| 및| 사회| 안전|망| 강화|의| 필요|성을| 가져|옵|니다|.

|5|.| **|정|신| 건강| 문제|**|:| 스트|레스|와| 경쟁|,| 사회|적| 고|립| 등|으로| 인해| 정신| 건강| 문제가| 빈|번|해|지고| 있으며|,| 이에| 대한| 인|식|과| 지원|이| 부족|한| 상황|입니다|.

|6|.| **|성| 차|별| 및| 성|폭|력|**|:| 성|별|에| 따른| 차|별|과| 성|폭|력| 사건|이| 여|전히| 사회| 문제|로| 남|아| 있으며|,| 이에| 대한| 인|식| 및| 법|적| 대응|이| 미|흡|한| 경우|가| 많|습니다|.

|이| 외|에도| 여러| 가지| 문제가| 있지만|,| 각각|의| 문제|들은| 상|호| 연결|되어| 있으며| 해결|하기| 위해|서는| 종|합|적인| 접근

In [ ]:
# tool 있을 때 스트림 출ㄹ력
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?"),
]

response = llm_with_tools.stream(messages)

# 파편화된 tool_call 청크를 하나로 합치기 
is_first = True
for chunk in response:    
    print("chunk type: ", type(chunk))
    
    if is_first:
        is_first = False
        gathered = chunk
    else:
        gathered += chunk
    
    print("content: ", gathered.content, "tool_call_chunk", gathered.tool_calls)

messages.append(gathered)


chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_7XS8gPM2SSeat3NnJhCstDpG', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_7XS8gPM2SSeat3NnJhCstDpG', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_7XS8gPM2SSeat3NnJhCstDpG', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': ''}, 'id': 'call_7XS8gPM2SSeat3NnJhCstDpG', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': 'Asia'}, 'id': 'call_7XS8gPM2SSeat3NnJhCstDpG', 'type': 'tool_c

In [17]:
# AIMessageChunk는 + 연산자로 계속 이어 붙일 수 있는 형태로 유지
gathered

AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_7XS8gPM2SSeat3NnJhCstDpG', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"부산"}', 'name': 'get_current_time'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'service_tier': 'default'}, id='run--1de885cd-402e-443e-9537-8e7321dde245', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_7XS8gPM2SSeat3NnJhCstDpG', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_current_time', 'args': '{"timezone":"Asia/Seoul","location":"부산"}', 'id': 'call_7XS8gPM2SSeat3NnJhCstDpG', 'index': 0, 'type': 'tool_call_chunk'}])

In [19]:
# AIMessageChunk에 기존 대화를 추가해서 도구 사용하기
for tool_call in gathered.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul (부산) 현재시각 2025-07-03 11:18:46 


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_7XS8gPM2SSeat3NnJhCstDpG', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"부산"}', 'name': 'get_current_time'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'service_tier': 'default'}, id='run--1de885cd-402e-443e-9537-8e7321dde245', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_7XS8gPM2SSeat3NnJhCstDpG', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_current_time', 'args': '{"timezone":"Asia/Seoul","location":"부산"}', 'id': 'call_7XS8gPM2SSeat3NnJhCstDpG', 'index': 0, 'type': 'tool_call_chunk'}]),
 ToolMessage(content='Asia/S

In [20]:
for c in llm_with_tools.stream(messages):
    print(c.content, end="|")

|부|산|은| 현재| |202|5|년| |7|월| |3|일| |11|시| |18|분| |46|초|입니다|.||